In [1]:
import requests
import xmltodict
import datetime
import pandas as pd
import glob
import re

In [2]:
# 현재 날짜 확인
now = datetime.datetime.now()
nowDate = now.strftime('%Y%m%d')

nowDate

'20210531'

In [3]:
# 현재 디렉토리에 있는 가장 최신 파일 날짜 확인
ls_all = glob.glob("./dataset/all/*.csv")
if not ls_all:
    oldDate_all = '20200101'
else:
    ls_all.sort()
    oldDate_all = ls_all[-1][22:-8]
    oldDate_csv_all = ls_all[-1][14:]

ls_part = glob.glob("./dataset/part/*.csv")
if not ls_part:
    oldDate_part = '20200101'
else:
    ls_part.sort()
    oldDate_part = ls_part[-1][23:-9]
    oldDate_csv_part = ls_part[-1][15:]

oldDate_all, oldDate_csv_all, oldDate_part, oldDate_csv_part

# all: 전국 통합, part: 시, 도 분할

('20210524',
 'covid19_20210524_all.csv',
 '20210524',
 'covid19_20210524_part.csv')

In [4]:
# 현재 날짜부터 디렉토리에 있는 가장 최신 파일 날짜까지 데이터 다운로드
url_base_all = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'
url_base_part = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson'
url_serviceKey = 'j8RHAoYLh5NFcdYjbRbdSoOGiuOeHwQZZaU3Mqu78SUYGddoObBnh0Xx3o%2BVPFtZhz5hZaQOf%2B9dMFljJK%2BpLQ%3D%3D'
url_pages = '1000'
url_start_date_all = oldDate_all
url_start_date_part = oldDate_part
url_end_date = nowDate
url_all = url_base_all + '?serviceKey=' + url_serviceKey + '&pageNo=1&numOfRows=' + url_pages + '&startCreateDt=' + url_start_date_all + '&endCreateDt=' + url_end_date
url_part = url_base_part + '?serviceKey=' + url_serviceKey + '&pageNo=1&numOfRows=' + url_pages + '&startCreateDt=' + url_start_date_part + '&endCreateDt=' + url_end_date

In [5]:
req_all = requests.get(url_all).content
req_part = requests.get(url_part).content
xmlObject_all = xmltodict.parse(req_all)
xmlObject_part = xmltodict.parse(req_part)
dict_data_all = xmlObject_all['response']['body']['items']['item']
dict_data_part = xmlObject_part['response']['body']['items']['item']

In [6]:
df_new_all = pd.DataFrame(dict_data_all)
df_new_part = pd.DataFrame(dict_data_part)

In [7]:
for i in range(df_new_part.shape[0]):
    t = re.split(r'년|월|일|시| ', df_new_part['stdDay'][i])
    df_new_part['stdDay'][i] = ''.join(t[0:5:2])

df_new_part

,createDt,deathCnt,defCnt,gubun,gubunCn,gubunEn,incDec,isolClearCnt,isolIngCnt,localOccCnt,overFlowCnt,qurRate,seq,stdDay,updateDt
0,2021-05-31 09:41:22.919,4,3844,검역,隔離區,Lazaretto,7,3600,240,0,7,-,9849,20210531,null
1,2021-05-31 09:41:22.918,1,1030,제주,济州,Jeju,7,898,131,6,1,153.56,9848,20210531,null
2,2021-05-31 09:41:22.918,19,4699,경남,庆南,Gyeongsangnam-do,16,4468,212,15,1,139.80,9847,20210531,null
3,2021-05-31 09:41:22.918,85,4692,경북,庆北,Gyeongsangbuk-do,18,4378,229,15,3,176.22,9846,20210531,null
4,2021-05-31 09:41:22.918,15,1463,전남,全南,Jeollanam-do,6,1282,166,6,0,78.46,9845,20210531,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,2021-05-24 09:46:24.359,61,6064,인천,仁川,Incheon,18,5803,200,18,0,205.13,9702,20210524,null
148,2021-05-24 09:46:24.358,221,9733,대구,大邱,Daegu,48,9228,284,48,0,399.47,9701,20210524,null
149,2021-05-24 09:46:24.358,124,5555,부산,釜山,Busan,15,5169,262,14,1,162.82,9700,20210524,null
150,2021-05-24 09:46:24.358,480,42601,서울,首尔,Seoul,139,39276,2845,136,3,437.67,9699,20210524,null


In [8]:
df_new_all = df_new_all.sort_values(by=['stateDt'])
df_new_part = df_new_part.sort_values(by=['stdDay'])
df_new_all = df_new_all.drop(df_new_all.index[0:1])
df_new_part = df_new_part.drop(df_new_part.index[0:20])

In [9]:
df_new_all

,accDefRate,accExamCnt,accExamCompCnt,careCnt,clearCnt,createDt,deathCnt,decideCnt,examCnt,resutlNegCnt,seq,stateDt,stateTime,updateDt
6,1.4455504748,9592727,9475629,8044,126993,2021-05-25 09:33:39.762,1938,136975,117098,9338654,522,20210525,00:00,2021-05-26 17:26:57.787
5,1.4479226274,9629450,9508381,8160,127574,2021-05-26 09:46:42.628,1940,137674,121069,9370707,523,20210526,00:00,2021-05-27 09:40:52.468
4,1.4493399059,9665693,9543034,8188,128180,2021-05-27 09:39:43.099,1943,138311,122659,9404723,524,20210527,00:00,null
3,1.4499377008,9702456,9579584,8193,128759,2021-05-28 09:34:29.553,1946,138898,122872,9440686,525,20210528,00:00,2021-05-29 09:47:35.466
2,1.4509249463,9733591,9609801,7741,129739,2021-05-29 09:45:26.765,1951,139431,123790,9470370,526,20210529,00:00,null
1,1.4541964496,9747614,9621121,7572,130381,2021-05-30 09:24:39.384,1957,139910,126493,9481211,527,20210530,00:00,null
0,1.4568360166,9761158,9633205,7558,130823,2021-05-31 09:34:29.254,1959,140340,127953,9492865,528,20210531,00:00,null


In [10]:
df_new_part
#   속성 설명(all)
#   DECIDE_CNT          확진자수
#   CLEAR_CNT           격리해제 수
#   EXAM_CNT            검사진행 수
#   DEATH_CNT           사망자 수
#   CARE_CNT            치료중 환자 수
#   RESULT_NEG_CNT      결과 음성 수
#   ACC_EXAM_CNT        누적 검사 수
#   ACC_EXAM_COMP_CNT   누적 검사 완료 수
#   ACC_DEF_RATE        누적 확진률
#   CREATE_DT           등록일시분초
#   UPDATE_DT           수정일시분

,createDt,deathCnt,defCnt,gubun,gubunCn,gubunEn,incDec,isolClearCnt,isolIngCnt,localOccCnt,overFlowCnt,qurRate,seq,stdDay,updateDt
114,2021-05-25 09:37:21.497,4,3787,검역,隔離區,Lazaretto,22,3538,245,0,22,-,9735,20210525,null
115,2021-05-25 09:37:21.497,1,963,제주,济州,Jeju,17,847,115,17,0,143.57,9734,20210525,null
116,2021-05-25 09:37:21.497,19,4594,경남,庆南,Gyeongsangnam-do,14,4331,244,13,1,136.67,9733,20210525,null
117,2021-05-25 09:37:21.496,84,4617,경북,庆北,Gyeongsangbuk-do,12,4262,271,11,1,173.41,9732,20210525,null
118,2021-05-25 09:37:21.496,13,1409,전남,全南,Jeollanam-do,7,1193,203,7,0,75.56,9731,20210525,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,2021-05-31 09:41:22.916,124,5668,부산,釜山,Busan,17,5340,204,16,1,166.13,9833,20210531,null
17,2021-05-31 09:41:22.916,488,43917,서울,首尔,Seoul,130,40815,2614,129,1,451.19,9832,20210531,null
18,2021-05-31 09:41:22.916,1959,140340,합계,合计,Total,430,130823,7558,411,19,270.68,9831,20210531,null
9,2021-05-31 09:41:22.917,638,39081,경기,京畿,Gyeonggi-do,130,36472,1971,127,3,294.94,9840,20210531,null


In [11]:
df_old_all = pd.read_csv('./dataset/all/covid19_' + oldDate_all + '_all.csv')
df_old_all = df_old_all.drop(['Unnamed: 0'], axis = 1)
df_old_part = pd.read_csv('./dataset/part/covid19_' + oldDate_part + '_part.csv')
df_old_part = df_old_part.drop(['Unnamed: 0'], axis = 1)
df_old_all

,accDefRate,accExamCnt,accExamCompCnt,careCnt,clearCnt,createDt,deathCnt,decideCnt,examCnt,resutlNegCnt,seq,stateDt,stateTime,updateDt
0,NaN,NaN,NaN,NaN,1,47:33.3,0,0,1,NaN,1,20200101,18:00,21:56.6
1,NaN,NaN,NaN,NaN,1,47:33.3,0,0,1,NaN,1,20200201,18:00,47:33.3
2,NaN,NaN,NaN,NaN,2,22:49.5,2,2,2,NaN,2,20200202,9:00,22:49.5
3,NaN,NaN,NaN,NaN,0,41:17.2,0,15,0,NaN,3,20200203,9:00,41:17.2
4,NaN,NaN,NaN,NaN,0,26:59.6,0,0,0,NaN,4,20200204,0:00,26:59.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,1.436175,9447358.0,9338485.0,8542.0,123659,2021-05-20 09:35:45.542,1916,134117,108873,9204368.0,517,20210520,00:00,2021-05-21 09:47:58.875
476,1.436877,9487108.0,9372964.0,8598.0,124158,2021-05-21 09:36:32.023,1922,134678,114144,9238286.0,518,20210521,00:00,NaN
477,1.439308,9520966.0,9403408.0,8386.0,125032,2021-05-22 09:37:01.885,1926,135344,117558,9268064.0,519,20210522,00:00,NaN
478,1.443893,9536299.0,9414064.0,8117.0,125881,2021-05-23 09:37:54.099,1931,135929,122235,9278135.0,520,20210523,00:00,NaN


In [12]:
df_all = df_old_all.append(df_new_all)
df_part = df_old_part.append(df_new_part)
df_all.to_csv('./dataset/all/covid19_' + nowDate + '_all.csv', mode='w')
df_part.to_csv('./dataset/part/covid19_' + nowDate + '_part.csv', mode='w')

In [13]:
df_all.info()















<class 'pandas.core.frame.DataFrame'>
Int64Index: 487 entries, 0 to 0
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   accDefRate      456 non-null    object
 1   accExamCnt      456 non-null    object
 2   accExamCompCnt  456 non-null    object
 3   careCnt         455 non-null    object
 4   clearCnt        487 non-null    object
 5   createDt        487 non-null    object
 6   deathCnt        487 non-null    object
 7   decideCnt       487 non-null    object
 8   examCnt         487 non-null    object
 9   resutlNegCnt    456 non-null    object
 10  seq             487 non-null    object
 11  stateDt         487 non-null    object
 12  stateTime       487 non-null    object
 13  updateDt        427 non-null    object
dtypes: object(14)
memory usage: 57.1+ KB
